In [1]:
import json

import tensorflow as tf

import numpy as np
import random
import pickle 
import matplotlib.pyplot as plt
from collections import Counter

 
import vist
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)


In [2]:
# with open("data/caltech_stories_dataset", 'rb') as file:
#         caltech_stories_dataset = pickle.load(file)
        

# type(caltech_stories_dataset)
# labels = [1] * 12000
# labels[7200:] = [0] * 4800

# caltech_stories_dataset = np.array(caltech_stories_dataset)
# labels = np.array(labels)

# #shuffle and create datasets
# indices = list(range(12000))
# random.shuffle(indices)

# labels = labels[indices]
# caltech_stories_dataset = caltech_stories_dataset[indices]


# train = {"features" : {"filenames" : caltech_stories_dataset[:10000,:]}, "labels" : labels[:10000]}
# valid = {"features" : {"filenames" : caltech_stories_dataset[10000:11000,:]}, "labels" : labels[10000:11000]}
# test = {"features" : {"filenames" : caltech_stories_dataset[11000:,:]}, "labels" : labels[11000:]}

# caltech_data = {"training": train, "validation": valid, "test" : test}
# print("Labeled and segmented the dataset into training/validation/test sets")

Labeled and segmented the dataset into training/validation/test sets


In [27]:
with open("data/caltech_stories_patterns", 'rb') as file:
        caltech_stories_dataset = pickle.load(file)
        
train = {"features" : {"filenames" : caltech_stories_dataset["data"]["train"]}, "labels" : caltech_stories_dataset["labels"]["train"]}
valid = {"features" : {"filenames" : caltech_stories_dataset["data"]["validation"]}, "labels" : caltech_stories_dataset["labels"]["validation"]}
test = {"features" : {"filenames" : caltech_stories_dataset["data"]["test"]}, "labels" : caltech_stories_dataset["labels"]["test"]}

caltech_data = {"training": train, "validation": valid, "test" : test}
print("Labeled and segmented the dataset into training/validation/test sets")

Labeled and segmented the dataset into training/validation/test sets


In [28]:
with open("data/caltech_features", 'rb') as file:
        caltech_features = pickle.load(file)
        
for _set in caltech_data:
    features = caltech_data[_set]["features"]
    filenames = features["filenames"]
    features["fc7_features"] = []
    features["fc8_features"] = []
    
    for story in filenames:
        
        fc7_features_story = []
        fc8_features_story = []
        
        for img_filename in story:
            tokens = img_filename.split("/")
            class_name = tokens[-2]
            img_name = tokens[-1] 
            
            fc7 = caltech_features[class_name][img_name]["fc7"]
            fc8 = caltech_features[class_name][img_name]["fc8"]
            
            fc7_features_story.append(fc7) 
            fc8_features_story.append(fc8)
        
        features["fc7_features"].append(fc7_features_story) 
        features["fc8_features"].append(fc8_features_story)

print("Loaded fc7 and fc8 features")

Loaded fc7 and fc8 features


In [29]:
print(caltech_data.keys())
print(caltech_data['training'].keys())
print(caltech_data['training']['features'].keys())

dict_keys(['validation', 'test', 'training'])
dict_keys(['labels', 'features'])
dict_keys(['filenames', 'fc8_features', 'fc7_features'])


In [38]:
import itertools  

def preprocess_data(caltech_data, set_name, features_name):
    features = caltech_data[set_name]['features'][features_name]
    features = [list(itertools.chain(*story_features)) for story_features in features]
    features = np.array(features)
    bin_label = caltech_data[set_name]['labels']
    
    labels = np.zeros([features.shape[0],2]) 
    labels[np.arange(features.shape[0]), np.array(bin_label).reshape(1,features.shape[0])] = 1
    
    return features, labels

train_features, train_labels = preprocess_data(caltech_data, "training", "fc8_features")
validation_features, validation_labels = preprocess_data(caltech_data, "validation", "fc8_features")
test_features, test_labels = preprocess_data(caltech_data, "test", "fc8_features")

In [41]:


def get_batch(X, Y, current_batch, batch_size):
    batch_x = X[current_batch*batch_size : (current_batch+1)*batch_size, :]
    batch_y = Y[current_batch*batch_size : (current_batch+1)*batch_size, :] 
    return batch_x, batch_y

# Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100
display_step = 1

# Network Parameters
n_input = 5000 # MNIST data input (img shape: 28*28)
n_hidden_1 = 512 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

keep_prob = tf.placeholder(tf.float32)


# Create model
def multilayer_perceptron(x, weights, biases, keep_prob):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
 


# Construct model
pred = multilayer_perceptron(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_features.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = get_batch(train_features, train_labels, i, batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y,
                                                          keep_prob: 0.5})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print(" Train accuracy:", accuracy.eval(feed_dict = {x: train_features, y: train_labels, keep_prob : 1.0}))
            
            print(" Validation accuracy:", accuracy.eval(feed_dict = {x: validation_features, y: validation_labels, keep_prob : 1.0}))
    print("Optimization Finished!")

    print()
    # Test model
    print("Test accuracy:", accuracy.eval(feed_dict ={x: test_features, y: test_labels, keep_prob : 1.0}))


Epoch: 0001 cost= 9669.034150391
 Train accuracy: 0.8533
 Validation accuracy: 0.858
Epoch: 0002 cost= 3937.475621338
 Train accuracy: 0.9125
 Validation accuracy: 0.904
Epoch: 0003 cost= 2736.971962891
 Train accuracy: 0.9366
 Validation accuracy: 0.919
Epoch: 0004 cost= 2033.279336243
 Train accuracy: 0.9486
 Validation accuracy: 0.941
Epoch: 0005 cost= 1648.463715210
 Train accuracy: 0.9553
 Validation accuracy: 0.936
Epoch: 0006 cost= 1385.356647491
 Train accuracy: 0.9589
 Validation accuracy: 0.935
Epoch: 0007 cost= 1180.105024414
 Train accuracy: 0.9674
 Validation accuracy: 0.944
Epoch: 0008 cost= 961.712300568
 Train accuracy: 0.9703
 Validation accuracy: 0.948
Epoch: 0009 cost= 906.258187027
 Train accuracy: 0.9744
 Validation accuracy: 0.945
Epoch: 0010 cost= 750.358678665
 Train accuracy: 0.9779
 Validation accuracy: 0.957
Optimization Finished!

Test accuracy: 0.96
